# Housekeeping

## Library imports

In [7]:
import pandas as pd

import warnings

## Settings

In [8]:
warnings.filterwarnings("ignore")

## Data imports

In [30]:
data = pd.read_csv('../data/raw/MAI3004_lucki_mpa411.txt', delim_whitespace=True)
metadata = pd.read_csv('../data/raw/MAI3004_lucki_metadata_safe.csv')

In [31]:
data

#mpa_vJun23_CHOCOPhlAnSGB_202403
clade_name                                         mpa411_MMHP_Lucki_0893 mpa411_MMHP_Lucki_0505 mpa411_MMHP_Lucki_0629 mpa411_MMHP_Lucki_0498 mpa411_MMHP_Lucki_0699 mpa411_MMHP_Lucki_0635 mpa411_MMHP_Lucki_0810 mpa411_MMHP_Lucki_0558 mpa411_MMHP_Lucki_0697 mpa411_MMHP_Lucki_0695 mpa411_MMHP_Lucki_1016 mpa411_MMHP_Lucki_0163 mpa411_MMHP_Lucki_0473 mpa411_MMHP_Lucki_0115 mpa411_MMHP_Lucki_0381 mpa411_MMHP_Lucki_0474 mpa411_MMHP_Lucki_0372 mpa411_MMHP_Lucki_0207 mpa411_MMHP_Lucki_0509 mpa411_MMHP_Lucki_0047 mpa411_MMHP_Lucki_0610 mpa411_MMHP_Lucki_0025 mpa411_MMHP_Lucki_0368 mpa411_MMHP_Lucki_0840 mpa411_MMHP_Lucki_0934 mpa411_MMHP_Lucki_0200 mpa411_MMHP_Lucki_0713 mpa411_MMHP_Lucki_0364 mpa411_MMHP_Lucki_0854 mpa411_MMHP_Lucki_0545 mpa411_MMHP_Lucki_0617 mpa411_MMHP_Lucki_0210 mpa411_MMHP_Lucki_0995 mpa411_MMHP_Lucki_0763 mpa411_MMHP_Lucki_0261 mpa411_MMHP_Lucki_0524 mpa411_MMHP_Lucki_0356 mpa411_MMHP_Lucki_0002 mpa411_MMHP_Lucki_0658 mpa411_MMHP_Lucki_0181 mpa411_MMHP_Lucki_0432 mpa411_MMHP_Lucki_0520 mpa411_MMHP_Lucki_0386 mpa411_MMHP_Lucki_0301 mpa411_MMHP_Lucki_0418 mpa411_MMHP_Lucki_0471 mpa411_MMHP_Lucki_1010 mpa411_MMHP_Lucki_0848 mpa411_MMHP_Lucki_0865 mpa411_MMHP_Lucki_0522 mpa411_MMHP_Lucki_0919 mpa411_MMHP_Lucki_1048 mpa411_MMHP_Lucki_0786 mpa411_MMHP_Lucki_0519 mpa411_MMHP_Lucki_0580 mpa411_MMHP_Lucki_0149 mpa411_MMHP_Lucki_0511 mpa411_MMHP_Lucki_0917 mpa411_MMHP_Lucki_0769 mpa411_MMHP_Lucki_0348 mpa411_MMHP_Lucki_0028 mpa411_MMHP_Lucki_0816 mpa411_MMHP_Lucki_0898 mpa411_MMHP_Lucki_0291 mpa411_MMHP_Lucki_0899 mpa411_MMHP_Lucki_0513 mpa411_MMHP_Lucki_0708 mpa411_MMHP_Lucki_0271 mpa411_MMHP_Lucki_0185 mpa411_MMHP_Lucki_0199 mpa411_MMHP_Lucki_0323 mpa411_MMHP_Lucki_0344 mpa411_MMHP_Lucki_0208 mpa411_MMHP_Lucki_0768 mpa411_MMHP_Lucki_0921 mpa411_MMHP_Lucki_0621 mpa411_MMHP_Lucki_0868 mpa411_MMHP_Lucki_0711 mpa411_MMHP_Lucki_0420 mpa411_MMHP_Lucki_0716 mpa411_MMHP_Lucki_0343 mpa411_MMHP_Lucki_0328 mpa411_MMHP_Lucki_0795 mpa411_MMHP_Lucki_0501 mpa411_MMHP_Lucki_1005 mpa411_MMHP_Lucki_1066 mpa411_MMHP_Lucki_1007 mpa411_MMHP_Lucki_1075 mpa411_MMHP_Lucki_0642 mpa411_MMHP_Lucki_0022 mpa411_MMHP_Lucki_0500 mpa411_MMHP_Lucki_0930 mpa411_MMHP_Lucki_0465 mpa411_MMHP_Lucki_0355 mpa411_MMHP_Lucki_0507 mpa411_MMHP_Lucki_0295 mpa411_MMHP_Lucki_0156 mpa411_MMHP_Lucki_0907 mpa411_MMHP_Lucki_0083 mpa411_MMHP_Lucki_0260 mpa411_MMHP_Lucki_0926 mpa411_MMHP_Lucki_0821 mpa411_MMHP_Lucki_0068 mpa411_MMHP_Lucki_0394 mpa411_MMHP_Lucki_0872 mpa411_MMHP_Lucki_0170 mpa411_MMHP_Lucki_0784 mpa411_MMHP_Lucki_0516 mpa411_MMHP_Lucki_0493 mpa411_MMHP_Lucki_0743 mpa411_MMHP_Lucki_0549 mpa411_MMHP_Lucki_0977 mpa411_MMHP_Lucki_0373 mpa411_MMHP_Lucki_0637 mpa411_MMHP_Lucki_0762 mpa411_MMHP_Lucki_0887 mpa411_MMHP_Lucki_0194 mpa411_MMHP_Lucki_0623 mpa411_MMHP_Lucki_0166 mpa411_MMHP_Lucki_0828 mpa411_MMHP_Lucki_0529 mpa411_MMHP_Lucki_0187 mpa411_MMHP_Lucki_0804 mpa411_MMHP_Lucki_0806 mpa411_MMHP_Lucki_0327 mpa411_MMHP_Lucki_0120 mpa411_MMHP_Lucki_0238 mpa411_MMHP_Lucki_0314 mpa411_MMHP_Lucki_0714 mpa411_MMHP_Lucki_0808 mpa411_MMHP_Lucki_0073 mpa411_MMHP_Lucki_0086 mpa411_MMHP_Lucki_0244 mpa411_MMHP_Lucki_0688 mpa411_MMHP_Lucki_0428 mpa411_MMHP_Lucki_0984 mpa411_MMHP_Lucki_0139 mpa411_MMHP_Lucki_0894 mpa411_MMHP_Lucki_1070 mpa411_MMHP_Lucki_0094 mpa411_MMHP_Lucki_0413 mpa411_MMHP_Lucki_0329 mpa411_MMHP_Lucki_0823 mpa411_MMHP_Lucki_0382 mpa411_MMHP_Lucki_0559 mpa411_MMHP_Lucki_0742 mpa411_MMHP_Lucki_0857 mpa411_MMHP_Lucki_0829 mpa411_MMHP_Lucki_0153 mpa411_MMHP_Lucki_0438 mpa411_MMHP_Lucki_0931 mpa411_MMHP_Lucki_0771 mpa411_MMHP_Lucki_0476 mpa411_MMHP_Lucki_0574 mpa411_MMHP_Lucki_0007 mpa411_MMHP_Lucki_0573 mpa411_MMHP_Lucki_0904 mpa411_MMHP_Lucki_1076 mpa411_MMHP_Lucki_0496 mpa411_MMHP_Lucki_0518 mpa411_MMHP_Lucki_0612 mpa411_MMHP_Lucki_0565 mpa411_MMHP_Lucki_0941 mpa411_MMHP_Lucki_0346 mpa411_MMHP_Lucki_0378 mpa411_MMHP_Lucki_0883 mpa411_MMHP_Lucki_0819 mpa411_MMHP_Lucki_0949 mpa411_MMHP_Lucki_0540 mpa411_MMHP_Lucki_0309 mpa411

In [15]:
metadata

,sample_id,family_id,sex,body_product,age_group_at_sample,year_of_birth
0,MMHP_Lucki_0001,ENA_fam_0139,female,stool,1-2 weeks,2023
1,MMHP_Lucki_0002,ENA_fam_0139,female,stool,NaN,1995
2,MMHP_Lucki_0003,ENA_fam_0136,female,stool,1-2 weeks,2021
3,MMHP_Lucki_0004,ENA_fam_0136,female,stool,NaN,1978
4,MMHP_Lucki_0005,ENA_fam_0136,female,stool,4 weeks,2021
...,...,...,...,...,...,...
925,MMHP_Lucki_1072,ENA_fam_0001,female,stool,1-2 weeks,2023
926,MMHP_Lucki_1073,ENA_fam_0001,female,stool,4 months,2023
927,MMHP_Lucki_1074,ENA_fam_0001,female,stool,5 months,2023
928,MMHP_Lucki_1075,ENA_fam_0007,female,stool,1-2 weeks,2021


## Function definitions
| Function Name | Description | Parameters |
|---------------|-------------|------------|


# Data preprocessing